## Preprocessing

In [46]:
!pip install keras-tuner --upgrade
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [48]:
## Answer the following questions:
# Question 1: What variable(s) are the target(s) for your model?
# Answer: The taget variable for the model is 'IS_SUCCESSFUL' column from application_df; "was the money used effectively?"

# Question 2: What variable(s) are the are the feature(s) for your model?
# Answer: The feature variables for the model are all the other columns from application_df that ARE NOT the target variable ('IS_SUCCESSFUL'), or identification variables ('EIN' and 'NAME').

In [49]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [50]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [51]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [52]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [53]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [54]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = classification_counts.loc[classification_counts > 1]
classification_counts_gt1.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [55]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [56]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)

In [57]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=58)

In [58]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [59]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                440       
                                                                 
 dense_4 (Dense)             (None, 5)                 55        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 501 (1.96 KB)
Trainable params: 501 (1.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 1.5639 - accuracy: 0.6944
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.9029 - accuracy: 0.7140
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.8519 - accuracy: 0.7198
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.7634 - accuracy: 0.7146
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6543 - accuracy: 0.7190
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6077 - accuracy: 0.7231
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6014 - accuracy: 0.7248
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5880 - accuracy: 0.7262
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5761 - accuracy: 0.7271
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5737 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7336
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7342
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7344
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5446 - accuracy: 0.7341
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7340
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5445 - accuracy: 0.7351
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5458 - accuracy: 0.7332
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5456 - accuracy: 0.7333
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5444 - accuracy: 0.7341
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5443 - accuracy: 

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5749 - accuracy: 0.7273 - 787ms/epoch - 3ms/step
Loss: 0.5749414563179016, Accuracy: 0.7273469567298889


In [62]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")
